In [1]:
###aUXIALIARY FILE

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os, sys


sys.path.insert(1, "./../util")
sys.path.insert(1, "./../model")
from encoded_protein_dataset_new import get_embedding, EncodedProteinDataset_new, EncodedProteinDataset_aux, collate_fn_new#, dynamic_collate_fn
from pseudolikelihood import get_npll2, get_npll3
import torch, torchvision
from potts_decoder import PottsDecoder
from torch.utils.data import DataLoader, RandomSampler
from functools import partial

from typing import Sequence, Tuple, List
import scipy
from tqdm import tqdm
import pandas as pd
import csv
import time


##Lucas computer
sys.path.insert(1, "/home/luchinoprince/Dropbox/Old_OneDrive/Phd/Second_year/research/Feinauer/esm/")
import esm
#from esm.inverse_folding import util
import esm.pretrained as pretrained
from ioutils import read_fasta, read_encodings
from torch.nn.utils.rnn import pad_sequence
from collections import defaultdict
from Bio import SeqIO
from dynamic_loader import dynamic_collate_fn, dynamic_cluster
from torch.nn.functional import one_hot

from torch.nn.functional import softmax
from torch.distributions import Categorical


In [2]:
def compute_covariance(msa, q):
    """
    Compute covariance matrix of a given MSA having q different amino acids
    """
    M, N = msa.shape

    # One hot encode classes and reshape to create data matrix
    D = torch.flatten(one_hot(msa, num_classes=q), start_dim=1).to(torch.float32)

    # Remove one amino acid
    D = D.view(M, N, q)[:, :, :q-1].flatten(1)

    # Compute bivariate frequencies
    bivariate_freqs = D.T @ D / M
    
    # Compute product of univariate frequencies
    univariate_freqs = torch.diagonal(bivariate_freqs).view(N*(q-1), 1) @ torch.diagonal(bivariate_freqs).view(1, N*(q-1))

    return bivariate_freqs - univariate_freqs

q=21
embeddings = get_embedding(q)

In [3]:
max_msas = None
msa_dir = "/home/luchinoprince/split2/"
encoding_dir ="/media/luchinoprince/b1715ef3-045d-4bdf-b216-c211472fb5a2/Data/InverseFolding/structure_encodings/"

structure_test_dataset = EncodedProteinDataset_aux(os.path.join(msa_dir, 'test/structure'), encoding_dir, noise=0.0, max_msas=max_msas)
batch_structure_size_train = 1### I think with empty GPU we can go up to 16 easily
batch_structure_size=1
perc_subset_test = 1.0     ## During the training, for every dataset available we select a random 10% of its samples
batch_msa_size = 128 ### old is 32, original is 16
q = 21 ##isn't always 21
#dynamic_collate_fn

collate_fn = partial(dynamic_collate_fn, q=q, batch_size=batch_structure_size, batch_msa_size=batch_msa_size)
superfamily_test_loader = DataLoader(structure_test_dataset, batch_size=batch_structure_size, collate_fn=collate_fn, shuffle=False, 
num_workers=1, pin_memory=True)

/home/luchinoprince/Dropbox/Old_OneDrive/Phd/Second_year/research/Feinauer/Inverse_Folding/tests/./../util/encoded_protein_dataset_new.py:163: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  encodings = torch.tensor(read_encodings(encoding_path, trim=False))


/media/luchinoprince/b1715ef3-045d-4bdf-b216-c211472fb5a2/Data/InverseFolding/structure_encodings/4xhyA00.encodings.pt Mismatch in dimension, skipping /home/luchinoprince/split2/test/structure/4xhyA00.a3m.pt


In [4]:
device='cpu'
## This would work on windowsA
#bk_dir= 'D:/Data/InverseFolding/Intermediate_Models/'

bk_dir = "/media/luchinoprince/b1715ef3-045d-4bdf-b216-c211472fb5a2/Data/InverseFolding/Intermediate_Models/"
#fname_par = 'model_11_07_2023_epoch_' + str(94.0) + '_ardca' + '.pt'
fname_par = 'model_25_06_2023_epoch_' + str(94.0) + '.pt'
checkpoint = torch.load(os.path.join(bk_dir, fname_par))


q=21
args = checkpoint['args_run']
n_layers = args['n_layers']
param_embed_dim = d_model = args['param_embed_dim']
input_encoding_dim = args['input_encoding_dim']
n_heads=args['n_heads']
n_param_heads=args['n_param_heads']
dropout=args['dropout']

decoder = PottsDecoder(q, n_layers, d_model, input_encoding_dim, param_embed_dim, n_heads, n_param_heads, dropout=dropout).to(device);
#decoder.to(device);

decoder.load_state_dict(checkpoint['model_state_dict']);
decoder.eval();   ##to generate data we need just the forward pass of the model!

In [5]:
Ns = torch.zeros(structure_test_dataset.__len__())
Ms = torch.zeros(structure_test_dataset.__len__())
corr_potts = torch.zeros(structure_test_dataset.__len__())
corr_ardca = torch.zeros(structure_test_dataset.__len__())
decoder.eval()

alphabet='ACDEFGHIKLMNPQRSTVWY-'
default_index = alphabet.index('-')
aa_index = defaultdict(lambda: default_index, {alphabet[i]: i for i in range(len(alphabet))})
aa_index_inv = dict(map(reversed, aa_index.items()))

resexp = {}


idx = 0
import warnings
warnings.filterwarnings("ignore")
with torch.no_grad():
    for inputs_packed in superfamily_test_loader:
        #if idx>4:
        #    break
        print(f"I am at index:{idx} out of {structure_test_dataset.__len__()}", end="\r")
        ### I have to understand how to recover the correct name of the file
        for inputs in inputs_packed[1]:
            msas, encodings, padding_mask  = [input.to(device, non_blocking=True) for input in inputs]
            B, M, N = msas.shape
            #if idx>5:
            #    break
            #print(f"encodings' shape{encodings.shape}, padding mask:{padding_mask.shape}")
            couplings, fields = decoder(encodings, padding_mask)
            test_msa=torch.load(structure_test_dataset.msas_paths[idx]).to(torch.int)
            M_full = test_msa.shape[0]
            
            ## Save the model
            ## In this folder I will save the model in consideration, I won't save all of them due to memory
            #auxiliary_model_dir = '/media/luchinoprince/b1715ef3-045d-4bdf-b216-c211472fb5a2/Data/InverseFolding/Auxiliary_Potts'
            auxiliary_model_dir = "/home/luchinoprince/Auxiliary_Data_bmdca/"
            #auxiliary_model_dir = "."

            #B,N,_ = rep_padded.shape
            with open(os.path.join(auxiliary_model_dir, "potts_couplings_fields.txt"), "w") as f:
                ## write J
                for i in range(N):
                    for j in range(i+1, N):
                        for aa1 in range(q):
                            for aa2 in range(q):
                                J_el = couplings[0, i*q+aa1, j*q+aa2].detach().to('cpu').item()
                                line = "J " + str(i) + " " + str(j) + " "+ str(aa1) + " " + str(aa2) + " " + str(J_el) +"\n"
                                f.write(line)
                
                ## write h
                for i in range(N):
                    for aa in range(q):
                        h_el = fields[0, i*q+aa1].detach().to('cpu').item()
                        line = "h " + str(i) + " " + str(aa) + " " + str(h_el) + "\n"
                        f.write(line)
            ###### SAMPLE
            auxiliary_model_dir = "/home/luchinoprince/Auxiliary_Data_bmdca/"
            out_dir = '/home/luchinoprince/Auxiliary_Data_bmdca/Auxiliary_Samples_Potts/'
            out_file = 'samplesexp.txt'
            samples_path = os.path.join(auxiliary_model_dir, "potts_couplings_fields.txt")
            ## I generate a number of samples equal to the MSA, which we know is filtered to have at least 2k samples
            M_aux = int(np.ceil(M_full/10))
            ## The ! creates a terminal command, to pass variable you need to put square brackets
            !(bmdca_sample -p {samples_path} -n {M_aux} -d {out_dir} -o {out_file} -c bmdca.config)
            
            file='samplesexp_numerical.txt'
            with open(os.path.join(out_dir,file), mode='r') as f:
                lines=f.readlines()

            char_seq = [] ##36 is the lenght of YAP

            for i in range(1, len(lines)):
                line = lines[i][0:-1].split(" ") ## I take out the end of file
                line_char = [aa_index_inv[int(idx)] for idx in line]
                char_seq.append(line_char)
                
            ## Now re-translate
            for prot_idx in range(len(char_seq)):
                for aa in range(len(char_seq[prot_idx])):
                    char_seq[prot_idx][aa] = aa_index[char_seq[prot_idx][aa]]
                    
            msa_t = torch.tensor(char_seq, dtype=torch.long)
            cov_potts = compute_covariance(msa_t, 21)
            test_msa = torch.tensor(test_msa, dtype=torch.long)
            cov_train = compute_covariance(test_msa, q)
            res = scipy.stats.pearsonr(cov_train.flatten(), cov_potts.flatten())

            Ns[idx] = N
            Ms[idx] = test_msa.shape[0]
            corr_potts[idx] = res[0]
            ###### I LOAD THE PARTIAL RESULTS SO THAT IF IT BLOCKS WE DON'T LOSE ALL THE WORK ###########
            resexp['Ms'] = Ms
            resexp['Ns'] = Ns
            resexp['corr_potts'] = corr_potts
            torch.save(resexp, "res_corr_potts_structure.pkl")
            ##### This way if the output gets cluttered I can still see where I am
            with open("tracker.txt", "w") as f:
                line = "fineshed " + str(idx) + "\n"
                f.write(line)
            idx+=1


initializing sampler... 0.0120802 sec

sampling model with mcmc... 0.289518 sec
updating mcmc stats with samples... 0.026971 sec
computing sequence energies and correlations... 0.041655 sec
increasing burn-in time to 150000
resampling...
writing temporary files
sampling model with mcmc... 0.33591 sec
updating mcmc stats with samples... 0.0227687 sec
computing sequence energies and correlations... 0.0460316 sec
writing final sequences... done
initializing sampler... 930.0539396 sec

sampling model with mcmc... 1.84426 sec
updating mcmc stats with samples... 0.105189 sec
computing sequence energies and correlations... 0.0715362 sec
writing final sequences... done
initializing sampler... 932.00683 sec

sampling model with mcmc... 23.8759 sec
updating mcmc stats with samples... 3.52362 sec
computing sequence energies and correlations... 0.154292 sec
increasing wait time to 1500
increasing burn-in time to 150000
resampling...
writing temporary files
sampling model with mcmc... 33.0569 sec
u

In [ ]:
res = {}
res['Ms'] = Ms
res['Ns'] = Ns
res['corr_potts'] = corr_potts

In [ ]:
#torch.save(res, "res_corr_potts_structure.pkl")